In [ ]:
from typing import Callable, Iterator, Sequence
import functools

import numpy as np
import scipy as sp
import matplotlib.pylab as plt

import jax.numpy as jnp
import jax.scipy as jsp
import jax
import flax.linen as nn
import optax
import haiku as hk
import chex
import tqdm

## Dataset generation

In [ ]:
def generate_checkerboard(*, num: int, rng: np.random.Generator) -> np.ndarray:
    # https://github.com/malbergo/stochastic-interpolants/blob/main/notebooks/checker-mirror.ipynb
    x1 = rng.uniform(size=num) * 4 - 2
    x2_ = rng.uniform(size=num) - rng.choice([0, 1], size=(num,)) * 2
    x2 = x2_ + (np.floor(x1) % 2)
    x = np.hstack([x1[:, None], x2[:, None]]) * 2
    return x

In [ ]:
X_train = generate_checkerboard(num=2000, rng=np.random.default_rng())
plt.plot(X_train[:, 0], X_train[:, 1], '.')

## Define MLP score function

Here, we use the `flax` [library](https://flax.readthedocs.io/en/latest/) to define a fully connected neural network that represents our score function.

In [ ]:
Activation = Callable[[jax.Array], jax.Array]

class MLP(nn.Module):
    
    features: Sequence[int]  # Number of neurons in each layer
    activation: Activation = nn.swish

    @nn.compact
    def __call__(self, x: jax.Array) -> jax.Array:
        for f in self.features[:-1]:
            x = nn.Dense(f)(x)
            x = self.activation(x)
        x = nn.Dense(self.features[-1])(x)
        return x

model = MLP(features=[64, 64, 2])
print(model.tabulate(jax.random.PRNGKey(0), np.zeros((1, 2))))

## Define the score matching loss

In [ ]:
# Loss is not divided by n?
# Investigate whether alway size n for last batch
def denoising_score_matching_loss(params: chex.ArrayTree, batch: jax.Array, key: chex.PRNGKey, std: float, k: int) -> float:
    n, m = batch.shape
    batch = jnp.tile(batch[None, :, :], (k, 1, 1))

    # Apply noise to datapoints
    noise = jax.random.normal(key, batch.shape) * std
    noised_batch = noise + batch

    # DSM loss: x_tilde + sigma^2*f(x_tilde) - x
    fs = model.apply(params, noised_batch.reshape(k * n, m)).reshape(k, n, m)
    loss = jnp.sum(jnp.square(noised_batch + std**2 * fs - batch)) / k

    return loss

## Train the model

In [ ]:
optimizer = optax.adam(learning_rate=1e-1)
prng_seq = hk.PRNGSequence(jax.random.PRNGKey(0))

@jax.jit
def do_batch_update(
    batch: jax.Array,
    params: chex.ArrayTree, 
    opt_state: optax.OptState,  # Optimizer state
    key: chex.PRNGKey,
) -> tuple[float, chex.ArrayTree, optax.OptState]:
    loss, grad = jax.value_and_grad(denoising_score_matching_loss)(params, batch, key, std=0.01, k=100)
    updates, opt_state = optimizer.update(grad, opt_state)
    params = optax.apply_updates(params, updates)
    return loss, params, opt_state


class BatchManager(Iterator[np.ndarray]):
    
    def __init__(
        self,
        data: np.ndarray,
        batch_size: int,
        key: chex.PRNGKey
    ):
        batch_size = min(batch_size, len(data))
        self._num_batches = len(data) // batch_size
        self._batch_idx = None
        self._batch_size = batch_size
        self._key = hk.PRNGSequence(key)
        self._data = data
        self._reset()

    @property
    def num_batches(self) -> int:
        return self._num_batches

    def _reset(self) -> None:
        self._perm = np.array(jax.random.permutation(next(self._key), np.arange(len(self._data))))
        self._batch_idx = 0

    def __next__(self) -> np.ndarray:
        assert self._batch_idx is not None
        assert self._batch_idx >= 0 and self._batch_idx < self._num_batches
        inds = self._perm[self._batch_idx * self._batch_size : (self._batch_idx + 1) * self._batch_size]
        batch = self._data[inds]
        self._batch_idx += 1
        if self._batch_idx >= self._num_batches:
            self._reset()
        return batch

In [ ]:
print("Devices:", jax.devices())

In [ ]:
params = model.init(next(prng_seq), X_train[:1, ...])
opt_state = optimizer.init(params)
bm = BatchManager(data=X_train, batch_size=128, key=next(prng_seq))

losses = []
for _ in tqdm.tqdm(range(10000)):
    batch = next(bm)
    loss, params, opt_state = do_batch_update(batch, params, opt_state, key=next(prng_seq))
    losses.append(loss)
losses = np.array(losses)

In [ ]:
window_size = 1000
window = np.ones(window_size) / window_size
ave_losses = np.convolve(losses, window, mode='valid')
plt.plot(ave_losses)

## Sample from the model

In [ ]:
@functools.partial(jax.jit, static_argnames=("num_steps",))
def langevin_sampling(
    params: chex.ArrayTree,
    key: chex.PRNGKey,
    step_size: float,
    initial_samples: jax.Array,
    num_steps: int,
) -> jax.Array:

    def scan_fn(carry, _):
        states, key = carry
        key, sk = jax.random.split(key)
        noise = jax.random.normal(sk, shape=states.shape)
        next_states = states + step_size * model.apply(params, states) + jnp.sqrt(2 * step_size) * noise
        return (next_states, key), None

    states = initial_samples
    (states, _), _ = jax.lax.scan(scan_fn, (states, key), jnp.arange(num_steps))
    return states

In [ ]:
samples = langevin_sampling(
    params,
    next(prng_seq),
    5e-3,
    2 * jax.random.normal(next(prng_seq), shape=(2000, 2)),
    1000)
plt.plot(samples[:, 0], samples[:, 1], '.')
plt.plot(X_train[:, 0], X_train[:, 1], 'o', alpha=0.2)

plt.xlim([-4, 4])
plt.ylim([-4, 4])